A 为对称正定矩阵

$$
\left(\begin{array}{cccc}
a_{11} & a_{12} & \cdots & a_{1 n} \\
a_{21} & a_{22} & \cdots & a_{2 n} \\
\vdots & \vdots & & \vdots \\
a_{n 1} & a_{n 2} & \cdots & a_{n n}
\end{array}\right)=\left[\begin{array}{cccc}
l_{11} & & & \\
l_{21} & l_{22} & & \\
\vdots & \vdots & \ddots & \\
l_{n 1} & l_{n 2} & \cdots & l_{n n}
\end{array}\right]\left[\begin{array}{cccc}
l_{11} & l_{21} & \cdots & l_{n 1} \\
& l_{22} & \cdots & l_{n 2} \\
& & \ddots & \vdots \\
& & & l_{n n}
\end{array}\right] .
$$


第一步

$$
\left[\begin{array}{l|l}
a_{11} & A_{21}^{\mathrm{T}} \\
\hline A_{21} & A_{22}
\end{array}\right]=\left[\begin{array}{cc}
l_{11} & \\
L_{21} & L_{22}
\end{array}\right]\left[\begin{array}{ll}
l_{11} & L_{21}^{\mathrm{T}} \\
& L_{22}^{\mathrm{T}}
\end{array}\right]=\left[\begin{array}{cc}
l_{11}^2 & l_{11} L_{21}^{\mathrm{T}} \\
l_{11} L_{21} & L_{21} L_{21}^{\mathrm{T}}+L_{22} L_{22}^{\mathrm{T}}
\end{array}\right]
$$


按顺序求出 $l_{11}, L_{21}$ 后，再得 $L_{22} L_{22}^T$ ，而 $L_{22} L_{22}^T$ 依然是对称正定的，继续按照第一步的思路计算。

In [3]:
import numpy as np

def reg_utm(A, b):
    """ 解上三角方程组 回代算法 """
    if np.any(np.diag(A) == 0):
        raise ValueError("Error: 对角线上有0元素，不能使用该函数")

    n = len(b)
    X = np.zeros(n)
    X[n - 1] = b[n - 1] / A[n - 1, n - 1]

    for k in range(n - 2, -1, -1):
        t = np.dot(A[k, k + 1:n], X[k + 1:n])
        X[k] = (b[k] - t) / A[k, k]

    return X

def gsem_column(A, b):
    """
    列主元消去法
    A : 系数矩阵
    b : 右端常数 [列向量]
    X : 求得的解向量
    Ae: 得到的上三角矩阵
    be: 对应的右端项
    """
    A1 = np.hstack((A, b.reshape(-1, 1)))  # 增广矩阵
    n = len(A)
    n1 = A1.shape[1]

    # 系数矩阵化为上三角
    for i in range(n - 1):
        # 找到当前列的主元
        s = np.argmax(np.abs(A1[i:n, i])) + i
        # 交换行
        A1[[i, s], :] = A1[[s, i], :]

        # 进行消元操作
        for k in range(i + 1, n):
            lik = A1[k, i] / A1[i, i]  # 算子
            A1[k, i:n1] -= lik * A1[i, i:n1]

    # 得到上三角后进行回代
    Ae = A1[:, :n]
    be = A1[:, n1 - 1]
    X = reg_utm(Ae, be)

    return X, Ae, be

def Cholesky_fac(A):
    """
    对称正定矩阵的 Cholesky 分解
    A = LL'
    输入:
        A: 对称正定矩阵
    输出:
        分解后的下三角矩阵 L
    """
    n = len(A)
    for i in range(n - 1):
        A[i, i] = np.sqrt(A[i, i])
        A[i + 1:n, i] = A[i + 1:n, i] / A[i, i]
        A[i + 1:n, i + 1:n] -= np.outer(A[i + 1:n, i], A[i + 1:n, i])

    A[n - 1, n - 1] = np.sqrt(A[n - 1, n - 1])
    L = np.tril(A)  # 返回下三角部分
    return L

# 测试具体矩阵
A2 = np.array([[10, 1, 2, 3, 4],
               [1, 9, -1, 2, -3],
               [2, -1, 7, 3, -5],
               [3, 2, 3, 12, -1],
               [4, -3, -5, -1, 15]], dtype=float)

b = np.array([12, -27, 14, -17, 12], dtype=float)

# 使用 gsem_column 进行求解
X, Ae, be = gsem_column(A2, b)

# 输出解
print("解 x:", X)

解 x: [ 1. -2.  3. -2.  1.]
